In [7]:
import numpy as np
import cv2
import math

class Canny:
    def __init__(self, img, Gaussian_kernal_size):
        self.img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 图像灰度化处理
        self.Gaussian_kernal_size = Gaussian_kernal_size
        self.img_filter = self.Gaussian_Filter()
        self.img_gradient, self.img_direction = self.Sobel_Gradient()
        self.img_NMS = self.NMS()
        self.img_binary_boundary = self.Binary_Boundary()

        print("高斯滤波后的降噪图片:\n", self.img_filter)
        print("非极大值抑制后的图片:\n", self.img_NMS)
        print("阈值处理后的图片:\n", self.img_binary_boundary)

        cv2.imshow('Binary Boundary', self.img_binary_boundary)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    def Gaussian_Filter(self):
        sigma = 0.5
        size = self.Gaussian_kernal_size  
        Gaussian_kernel = np.zeros((size, size)) # 创建高斯核
        new_size = size // 2
        x, y = np.mgrid[-new_size:new_size+1, -new_size:new_size+1]
        Gaussian_kernel = np.exp(-((x**2 + y**2) / (2 * sigma**2))) / (2 * np.pi * sigma**2)
        
        # 归一化处理，所有数加起来为1
        Gaussian_kernel = Gaussian_kernel / Gaussian_kernel.sum()
        # 对图像进行padding
        img_padded = np.pad(self.img, ((new_size, new_size), (new_size, new_size)), mode='constant', constant_values=0)
        img_filter = np.zeros_like(self.img)
        # 对图像进行高斯滤波
        for i in range(self.img.shape[0]):
            for j in range(self.img.shape[1]):
                img_filter[i, j] = np.sum(img_padded[i:i+size, j:j+size] * Gaussian_kernel)

        return img_filter

    def Sobel_Gradient(self):
        img_sobel_x = cv2.Sobel(self.img_filter, cv2.CV_64F, 1, 0, ksize=3) # 用sobel算子求x方向梯度
        img_sobel_y = cv2.Sobel(self.img_filter, cv2.CV_64F, 0, 1, ksize=3) # 用sobel算子求y方向梯度

        img_gradient = np.sqrt(img_sobel_x**2 + img_sobel_y**2) # 求梯度幅值
        img_direction = np.arctan2(img_sobel_y, img_sobel_x) * (180 / np.pi) % 180 #将方向转换为角度

        return img_gradient, img_direction

    def NMS(self):
        img_NMS = np.zeros_like(self.img_gradient) # 将图形尺寸信息赋予要计算的非极大值抑制函数图像

        angle = self.img_direction
        for i in range(1, self.img_gradient.shape[0] - 1):
            for j in range(1, self.img_gradient.shape[1] - 1):
                q, r = 255, 255
                if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                    q = self.img_gradient[i, j + 1]
                    r = self.img_gradient[i, j - 1]
                elif 22.5 <= angle[i, j] < 67.5:
                    q = self.img_gradient[i + 1, j - 1]
                    r = self.img_gradient[i - 1, j + 1]
                elif 67.5 <= angle[i, j] < 112.5:
                    q = self.img_gradient[i + 1, j]
                    r = self.img_gradient[i - 1, j]
                elif 112.5 <= angle[i, j] < 157.5:
                    q = self.img_gradient[i - 1, j - 1]
                    r = self.img_gradient[i + 1, j + 1]

                if (self.img_gradient[i, j] >= q) and (self.img_gradient[i, j] >= r):
                    img_NMS[i, j] = self.img_gradient[i, j]
                else:
                    img_NMS[i, j] = 0

        return img_NMS


    def Binary_Boundary(self):
        Lower_threshold_value = self.img_gradient.mean() * 0.5 #通过像素平均值取低阈值
        Upper_threshold_value = Lower_threshold_value * 3 #设置高阈值为低阈值的3倍，经过试验比1.5倍效果好更清晰
        zhan = [] #利用栈，将所有高于高阈值的点进栈
        img_binary_boundary = np.copy(self.img_NMS) #创建最后的空图像尺寸
        '''
        1.首先遍历所有的点（剔除边界点），如果大于高阈值，进栈，小于低阈值，将其像素值设置为0
        '''
        for i in range(1, img_binary_boundary.shape[0] - 1):  
            for j in range(1, img_binary_boundary.shape[1] - 1):
                if img_binary_boundary[i, j] >= Upper_threshold_value:
                    img_binary_boundary[i, j] = 255
                    zhan.append([i, j])
                elif img_binary_boundary[i, j] <= Lower_threshold_value:
                    img_binary_boundary[i, j] = 0
        '''
        2.栈的长度不为0后，出栈，遍历在栈内的点周围八个点的像素，如果处于中间值，将其设置为255后进栈，循环遍历所有相连的点
        '''
        while len(zhan) != 0:
            x, y = zhan.pop()
            for di in [-1, 0, 1]:
                for dj in [-1, 0, 1]:
                    if di == 0 and dj == 0:
                        continue  
                    new_x, new_y = x + di, y + dj
                    if 0 <= new_x < img_binary_boundary.shape[0] and 0 <= new_y < img_binary_boundary.shape[1]:
                        if Lower_threshold_value < img_binary_boundary[new_x, new_y] < Upper_threshold_value:
                            img_binary_boundary[new_x, new_y] = 255
                            zhan.append([new_x, new_y])
    
        '''
        3.最后将散落的中间值点设置为0
        '''
        for i in range(img_binary_boundary.shape[0]):
            for j in range(img_binary_boundary.shape[1]):
                if img_binary_boundary[i, j] != 0 and img_binary_boundary[i, j] != 255:
                    img_binary_boundary[i, j] = 0
        return img_binary_boundary
    
    
if __name__ == '__main__':
    img = cv2.imread("lenna.png")
    Gaussian_kernal_size = 5
    new_img = Canny(img, Gaussian_kernal_size)

高斯滤波后的降噪图片:
 [[129 144 144 ... 150 137 104]
 [144 161 161 ... 168 153 117]
 [144 161 161 ... 168 153 117]
 ...
 [ 38  44  49 ... 103 100  88]
 [ 39  44  52 ... 103 104  95]
 [ 35  40  47 ...  92  93  85]]
非极大值抑制后的图片:
 [[  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.          93.33809512  68.         ...   0.         208.73428085
    0.        ]
 [  0.          68.           0.         ...   0.         204.
    0.        ]
 ...
 [  0.          43.10452412   0.         ...   0.          59.48108943
    0.        ]
 [  0.          50.69516742   0.         ...  40.52159918  47.20169488
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]]
阈值处理后的图片:
 [[  0.   0.   0. ...   0.   0.   0.]
 [  0. 255. 255. ...   0. 255.   0.]
 [  0. 255.   0. ...   0. 255.   0.]
 ...
 [  0. 255.   0. ...   0. 255.   0.]
 [  0. 255.   0. ... 255. 255.   0.]
 [  0.   0.   0. ...   0.   0.   0.]]
